In [1]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.nn import GELU
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import bisect

In [2]:
# Step 1: Define the Neural Network Architecture

# original: all fully connected 
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.GELU(),
            nn.Linear(128, 128),
            nn.GELU(),
            nn.Linear(128, output_size),
        )

    def forward(self, x):
        return self.net(x)
    
# updated model with conv1d layers
class PairedDualConvNet(nn.Module):
    def __init__(self, input_dim=101, output_dim=100, group_size=7, num_groups=7):
        super().__init__()
        assert input_dim >= 98, "Model assumes at least 98 inputs for paired operations."

        C1 = 8
        C2 = 8

        self.num_groups = num_groups        # 7 groups
        self.group_size = group_size      # each with 7 pairs
        self.num_pairs = num_groups*group_size            # first 98 inputs → 49 pairs
        self.paired_len = 2            # each pair has 2 elements
        self.used_for_pairing = self.num_pairs * self.paired_len
        self.leftover_dim = input_dim - self.used_for_pairing

        # -----------------------------
        # Stage 1: Conv over each pair
        # -----------------------------
        self.pair_conv = nn.Conv1d(1, C1, kernel_size=2)
        # Stage 1b: map C1 → 2 per pair
        self.pair_linear = nn.Conv1d(C1, 2, kernel_size=1)

        # -----------------------------
        # Stage 2: Branch A & B convs (2 → C2)
        # -----------------------------
        self.convA = nn.Conv1d(2, C2, kernel_size=group_size, padding=0)
        self.convB = nn.Conv1d(2, C2, kernel_size=num_groups, padding=0)

        # MLP input size: 49*C2 from branch A + 49*C2 from branch B + leftover features
        mlp_in = (num_groups * C2) + (group_size * C2) + self.leftover_dim
        self.fc1 = nn.Linear(mlp_in, 8)
        self.fc2 = nn.Linear(8, 64)
        self.fc3 = nn.Linear(64, output_dim)  # output 100 features

    def forward(self, x):
        B = x.shape[0]

        # -----------------------------
        # Split input
        # -----------------------------
        x_pairs = x[:, :self.used_for_pairing]   # (B, 98)
        x_leftover = x[:, self.used_for_pairing:]  # (B, 3)

        # -----------------------------
        # Stage 1: form pairs & conv
        # -----------------------------
        pairs = x_pairs.view(B, self.num_pairs, self.paired_len)  # (B, 49, 2)
        pairs_for_conv = pairs.reshape(B * self.num_pairs, 1, self.paired_len)  # (B*49,1,2)
        p = self.pair_conv(pairs_for_conv)  # (B*49, C1, 1)
        p = F.gelu(p)
        p = p.view(B, self.num_pairs, -1).permute(0, 2, 1)  # (B, C1, 49)

        # -----------------------------
        # Stage 1b: reduce 16 → 2 per pair
        # -----------------------------
        p = self.pair_linear(p)  # (B, 2, 49)
        p = F.gelu(p)

        # -----------------------------
        # Stage 2A: within-group-of-7 conv
        # -----------------------------
        num_groups = self.num_groups
        group_len = self.group_size
        p_groups = p.view(B, 2, num_groups, group_len)          # (B, 2, 7, 7)
        p_groups = p_groups.permute(0, 2, 1, 3).contiguous()    # (B,7,2,7)
        p_groups = p_groups.view(B*num_groups, 2, group_len)    # (B*7,2,7)
        a = self.convA(p_groups)                                 # (B*7, C2, 7)
        a = F.gelu(a)
        a = a.view(B, num_groups * a.shape[1])       # (B,7*C2)
        #a_flat = a.view(B, -1)                                   # (B, 49*C2)

        # -----------------------------
        # Stage 2B: spaced-by-7 conv
        # -----------------------------
        p_threads = p.view(B, 2, num_groups, group_len)         # (B,2,7,7)
        p_threads = p_threads.permute(0, 3, 1, 2).contiguous()  # (B,7,2,7)
        p_threads = p_threads.view(B*group_len, 2, num_groups)  # (B*7,2,7)
        b = self.convB(p_threads)                                # (B*7, C2, 7)
        b = F.gelu(b)
        b = b.view(B, group_len * b.shape[1])        # (B,7*C2)
        #b_flat = b.view(B, -1)                                   # (B,49*C2)

        # -----------------------------
        # Concatenate and MLP
        # -----------------------------
        h = torch.cat([a, b, x_leftover], dim=1)       # (B, 1571)
        h = F.gelu(self.fc1(h))
        h = F.gelu(self.fc2(h))
        out = self.fc3(h)                                        # (B,100)
        return out

In [3]:
# Step 2: Prepare the Data ---------------------------------------------------------------------

# ------------------------
# Load info
# ------------------------
info_df = pd.read_csv("info.csv")
fs = info_df.iloc[0, 5]                  # sampling frequency (Hz)

# ------------------------
# Load baseline data
# ------------------------
baseline_df = pd.read_csv("baselinedata.csv")
baseline_time = baseline_df.iloc[:, 0].values            # time column (seconds)
baseline_data = baseline_df.iloc[:, 1:].values           # data columns

# ------------------------
# Load main data
# ------------------------
df = pd.read_csv("data.csv")
time = df.iloc[:, 0].values            # time column (seconds)
data = df.iloc[:, 1:].values           # data columns

# ------------------------
# Load events
# ------------------------
events_df = pd.read_csv("events.csv")
event_times = events_df.iloc[:, 0].values  # assume first column is event time in seconds
event_times = np.sort(event_times)         # ensure sorted

# For fast lookup using binary search
def count_events_in_window(t, window=0.2):
    """
    Count how many event_times fall in (t - window, t].
    Uses bisect for O(log n) search.
    """
    left = bisect.bisect_right(event_times, t - window)
    right = bisect.bisect_right(event_times, t)
    return right - left

# ------------------------
# Compute event count for each row
# ------------------------
event_counts = np.array([count_events_in_window(t) for t in time])
event_counts = event_counts.reshape(-1, 1)

# Append event_counts as an additional input feature
data_aug = np.hstack([data, event_counts])
# Now each input row has: [original data..., event_count]

# ------------------------
# Build input-output pairs using the _ ms rule
# ------------------------
dt_target = 0.003      # ms
dt_tol = 0.0005        # ±0.5 ms
drow_target = int(dt_target * fs)  # number of rows corresponding to 3 ms

inputs = []
outputs = []
"""
# --- baseline data ---
Nbl = len(baseline_df)
nbl = int(math.floor(.8 * Nbl))
baseline_data_aug = np.hstack([baseline_data, np.zeros((Nbl, 1))])
for i in range(nbl - drow_target):
    dt = baseline_time[i+drow_target] - baseline_time[i]
    if abs(dt - dt_target) <= dt_tol:
        inputs.append(baseline_data_aug[i])   # baseline input (zero event count)
        outputs.append(baseline_data[i+drow_target])    # output is ONLY the data (no event count)
"""
# --- main data ---
N = len(df)
for i in range(N - drow_target):
    dt = time[i+drow_target] - time[i]
    if abs(dt - dt_target) <= dt_tol:
        inputs.append(data_aug[i])   # augmented input with event_count
        outputs.append(data[i+drow_target])    # output is ONLY the data (no event count)
"""
# --- baseline data ---
for i in range(nbl, Nbl - drow_target):
    dt = baseline_time[i+drow_target] - baseline_time[i]
    if abs(dt - dt_target) <= dt_tol:
        inputs.append(baseline_data_aug[i])   # baseline input (zero event count)
        outputs.append(baseline_data[i+drow_target])    # output is ONLY the data (no event count)
"""

X = torch.tensor(inputs, dtype=torch.float32)
Y = torch.tensor(outputs, dtype=torch.float32)

print("Pairs created:", len(X))
print("Input shape :", X.shape)   # features + 1
print("Output shape:", Y.shape)

# -----------------------------------------------------------------------------------------------


Pairs created: 374997
Input shape : torch.Size([374997, 101])
Output shape: torch.Size([374997, 100])


/var/folders/63/rd6zg9vd5v710s38rhy7y8jc0000gn/T/ipykernel_3737/1909333865.py:86: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:256.)
  X = torch.tensor(inputs, dtype=torch.float32)


In [4]:
# Step 3: Initialize the Model, Loss Function, and Optimizer
test_size=0.2
batch_size = 64
input_size = X.shape[1] # set input_size equal to the width of tensor X
output_size = Y.shape[1] # set output_size equal to the width of tensor Y
model = PairedDualConvNet(input_size, output_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# train / test
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size, random_state=42)
train_N = int((1 - test_size) * len(X))
X_train = X[:train_N]
Y_train = Y[:train_N]
X_test = X[train_N:]
Y_test = Y[train_N:]

# Create TensorDatasets
train_dataset = TensorDataset(X_train, Y_train)
test_dataset = TensorDataset(X_test, Y_test)

# Create DataLoaders for batching
train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size)

In [5]:
# data baseline characteristics as reference for loss 
mean_y = Y_train.mean(dim=0)
std_y = Y_train.std(dim=0)
var_y = std_y ** 2
var_per_feat = np.var(Y_train.numpy(), axis=0)  # redundant?

In [6]:
# Step 4: Train the Model
train_size = len(train_loader.dataset)
steps_per_epoch = math.ceil(train_size / batch_size)
print("Train samples:", train_size)
print("Batch size:", batch_size)
print("Batches/epoch:", steps_per_epoch)

model.train()
num_epochs = 100 # max
patience = 10
best_val = float('inf')
no_improve = 0
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # --- train ---
    model.train()
    running_loss = 0.0
    for X_batch, Y_batch in train_loader:
        # Forward pass
        Y_pred = model(X_batch)
        loss = criterion(Y_pred, Y_batch)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)
    epoch_train_loss = running_loss / train_size
    train_losses.append(epoch_train_loss)

    # --- validate ---
    model.eval()
    val_running = 0.0
    with torch.no_grad():
        for X_val, Y_val in test_loader:   # use test_loader or a separate val_loader
            Y_val_pred = model(X_val)
            l = criterion(Y_val_pred, Y_val)
            val_running += l.item() * X_val.size(0)
    epoch_val_loss = val_running / len(test_loader.dataset)
    val_losses.append(epoch_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} — train_loss: {epoch_train_loss:.6f}, val_loss: {epoch_val_loss:.6f}")

    # --- Early stopping ---
    if epoch_val_loss < best_val - 1e-9:
        best_val = epoch_val_loss
        no_improve = 0
        # Optionally save best model:
        torch.save(model.state_dict(), "neural_network_pytorch.pth")
    else:
        no_improve += 1
        if no_improve >= patience:
            print(f"No improvement for {patience} epochs — stopping early at epoch {epoch+1}.")
            break

# After loop: plot train/val loss to inspect convergence
plt.plot(train_losses, label='train_loss')
plt.plot(val_losses, label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()
plt.show()

Train samples: 299997
Batch size: 64
Batches/epoch: 4688
Epoch 1/100 — train_loss: 5.147589, val_loss: 3.643855
Epoch 2/100 — train_loss: 4.244904, val_loss: 3.484006


KeyboardInterrupt: 

In [ ]:
# load best model
# model.load_state_dict(torch.load("neural_network_pytorch.pth"))

: 

In [ ]:
# Step 5: Evaluate the Model on Test Data
model.eval()
with torch.no_grad():
    total_loss = 0
    for X_batch, Y_batch in test_loader:
        Y_pred = model(X_batch)
        loss = criterion(Y_pred, Y_batch)
        total_loss += loss.item() * X_batch.size(0)  # sum up batch loss

    avg_loss = total_loss / len(test_dataset)
    print(f"Test Loss: {avg_loss:.4f}")

Y_test_np = Y_test.numpy()
with torch.no_grad():
    Y_pred_np = model(X_test).numpy()

X_all_np = X.numpy()
Y_all_np = Y.numpy()
with torch.no_grad():
    Y_pred_all_np = model(X).numpy()

Y_null_all_np = X.numpy()[:, :Y.shape[1]]
Y_null_test_np = X_test.numpy()[:, :Y.shape[1]]

MSE_per_feat = np.mean((Y_test_np - Y_pred_np) ** 2, axis=0)
MSE_per_feat_null = np.mean((Y_test_np - Y_null_test_np) ** 2, axis=0)
feats = np.arange(1, Y.shape[1]+1)
barwid = .35

plt.figure(figsize=(15,5))
plt.bar(feats - barwid, var_per_feat, width=barwid, label='Output Variance')
plt.bar(feats, MSE_per_feat_null, width=barwid, label='Null MSE')
plt.bar(feats + barwid, MSE_per_feat, width=barwid, label='Test MSE')
plt.xlabel('Output Feature')
plt.ylabel('Value')
plt.title('Output Feature Variance vs Test MSE')
plt.legend()
plt.show()

Test Loss: 3.0843


In [ ]:
# simulations 
simdur = int(0.2 * fs) # samples 
plotdomain = 1000 * np.array([-1, 1]) + train_N

Ysim = []
i0 = plotdomain[0]
model.eval()
while i0+simdur < plotdomain[1]:
    xi = torch.tensor(X_all_np[i0, :].reshape(1, -1), dtype=torch.float32)
    for i in range(simdur):
        with torch.no_grad():
            yi = model(xi).numpy().flatten()
        Ysim.append(yi)
        # prepare next input
        if i < simdur - 1:
            event_count_next = X_all_np[i0 + i + 1, -1]  # keep using original event count
            xi = torch.tensor(np.hstack([yi, event_count_next]).reshape(1, -1), dtype=torch.float32)
    i0 += simdur
    print("Simulating:", (i0-plotdomain[0])/(plotdomain[1]-plotdomain[0]), " complete." )

Ysim = np.array(Ysim)
plotxval = np.arange(len(Ysim)) + plotdomain[0]

In [ ]:
# show several examples 

iMSE = np.argsort(MSE_per_feat)
iVAR = np.argsort(var_per_feat)
iLRN = np.argsort(MSE_per_feat / var_per_feat)
iToPlot = [iMSE[:2], iMSE[-2:], iVAR[:2], iVAR[-2:], iLRN[:2], iLRN[-2:]]
iToPlot = list(set([i for sublist in iToPlot for i in sublist]))

plt.figure(figsize=(15,20))
iPlot = 1
for i in iToPlot:
    plt.subplot(len(iToPlot), 1, iPlot)
    plt.plot(Y_all_np[:, i], label='True')
    plt.plot(Y_pred_all_np[:, i], label='Predicted', linestyle='--')
    plt.plot(Y_null_all_np[:, i], label='Null', linestyle=':')
    plt.plot(plotxval, Ysim[:,i], label='Simulated', linestyle='-.')
    plt.xlim(plotdomain)

    # set the y limits to be slightly larger than the min/max of true values in the plotdomain
    y_min = np.min(Y_all_np[plotdomain[0]:plotdomain[1], i])
    y_max = np.max(Y_all_np[plotdomain[0]:plotdomain[1], i])
    y_range = y_max - y_min
    plt.ylim(y_min - 0.1 * y_range, y_max + 0.1 * y_range)
    
    plt.title(f'Feature {i+1} - MSE: {MSE_per_feat[i]:.4f}, VAR: {var_per_feat[i]:.4f}')
    plt.legend(loc='upper right')
    iPlot += 1
plt.tight_layout()
plt.show()